<a href="https://colab.research.google.com/github/jesusGS01/TGINE/blob/main/P2_5_3_TGINE_JesusGarciaSalmeron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 2 : Análisis de información textual en YouTube

Alumno: Jesús García Salmerón

Convocatoria: Enero, 2024

## Parte 5.3 - Aplicar el uso de Transformers

In [1]:
# Instalamos las librerias necesarias
!pip3 install -U transformers datasets torch
!pip3 install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [2]:
import transformers
from transformers import AutoModelForSequenceClassification

# Ruta del modelo en Hugging Face
RoBERTa_spanish = 'dccuchile/albert-base-spanish'

# Numero de etiquetas que son 3 (Cocina, Deportes y Viaje)
NUM_LABELS = 3

# Cargamos el modelo para clasificación en Pytorch
bert_class_model_pytorch = AutoModelForSequenceClassification.from_pretrained(RoBERTa_spanish, num_labels=NUM_LABELS)
# Cargamos el tokenizer de este modelo
tokenizer = transformers.AutoTokenizer.from_pretrained(RoBERTa_spanish)

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/47.8M [00:00<?, ?B/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at dccuchile/albert-base-spanish and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

Procedemos a subir el fichero .zip con los datos a utilizar

In [3]:
from google.colab import files
# Subimos el fichero .zip
uploaded = files.upload()

Saving channelsData.zip to channelsData.zip


Extraemos el fichero .zip que hemos subido

In [4]:
import zipfile

# Ruta en la que se encuenta el .zip
zip_file_path = 'channelsData.zip'
# Ruta en la que vamos a descomprimir su contenido
extracted_folder_path = 'channelsData/'

# Descomprimimos su contenidos
try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_folder_path)
    print("Archivo ZIP descomprimido correctamente.")
except zipfile.BadZipFile:
    print("El archivo no es un ZIP válido.")

Archivo ZIP descomprimido correctamente.


Realizamos la creación de los distintos conjuntos de datos para "train", "validation" y "test"

In [8]:
import json
import os
from sklearn.model_selection import train_test_split
import pandas as pd

# Train
train_data = []
train_labels = []
# Test
test_data = []
test_labels = []

# Directorio padre
directory = 'channelsData'

# Contadores para saber el numero de canales pertenecientes a cada categoria
count_cocina = 0
count_deportes = 0
count_viaje = 0

# Recorremos los directorios del directorio padre 'channelsData'
for root, dirs, files in os.walk(directory):
    # Recorremos cada fichero de los subdirectorios
    for file in files:
        # Calculamos la ruta del fichero
        file_path = os.path.join(root, file)
        # Leemos el fichero y su información
        with open(file_path, 'r') as f:
            data = json.load(f)
            # Extraemos la temática del canal
            data_type = data['type']

            # Realizaremos para CADA TEMÁTICA, el 80% de los canales
            # para train y el 20% para test, es decir, que 8 canales
            # por temática serán para entrenamiento y 2 canales para test.
            # Esto será posible gracias a los contadores previos y una variable
            # que será para referir a la temática acutal.

            # En caso de que sea "Cocina"
            if data_type == "Cocina":
              count_cocina += 1
              actual_type = count_cocina
            # En caso de que sea "Deportes"
            elif data_type == "Deportes":
              count_deportes += 1
              actual_type = count_deportes
            # En caso de que sea "Viaje"
            elif data_type == "Viaje":
              count_viaje += 1
              actual_type = count_viaje

            # Extraemos la información de los distintos videos del canal
            data_videos = data['videos']
            # Recorremos los distintos videos extraidos
            for video in data_videos:
              # En caso de que no tengamos 7 canales para esa temática, será utilizado
              # este canal para train.
              if actual_type <= 8:
                train_data.append(video['description'])
                train_labels.append(data['type'])
              # En el caso contrario, será utilizado para test
              else:
                test_data.append(video['description'])
                test_labels.append(data['type'])

# Creamos el DataFrame con los datos de entrenamiento y su respectiva columna
df_train = pd.DataFrame(data=train_data, columns=['description'])
# Agregamos la columna de etiquetas al DataFrame de entrenamiento
df_train['label'] = train_labels

# Creamos el DataFrame con los datos de evaluación y su respectiva columna
df_test = pd.DataFrame(data=test_data, columns=['description'])
# Agregamos la columna de etiquetas al DataFrame de evaluación
df_test['label'] = test_labels

# Definimos un diccionario que mapee las etiquetas a números ya que necesitamos
# las diferentes clases como números
label_mapping = {"Cocina": 0, "Deportes": 1, "Viaje": 2}

# Aplicamos la transformación de las etiquetas en los DataFrames
df_train['label'] = df_train['label'].map(label_mapping)
df_test['label'] = df_test['label'].map(label_mapping)

# Creamos el DataFrame de validacion
p_eval = 0.2
# Dividimos el DataFrame de entrenamiento en "train" y "validation".
# El porcentaje a utilizar será de un 80% para train y un 20% para validar.
# Realizaremos una división estratificada, para asegurar que se manteniene
# la misma proporción de clases en el conjunto para validar.
# Establecemos también semilla de la división con 123.
df_training, df_eval = train_test_split (df_train, test_size = p_eval, stratify = df_train['label'], random_state = 123)

# Mostramos las longitud de los conjuntos de datos
print("Ejemplos usados para entrenar: ", len(df_training))
print("Ejemplos usados para evaluar: ", len(df_eval))
print("Ejemplos usados para test: ", len(df_test))

# Mostramos un poco del conjunto de evaluación.
df_eval.head()

Ejemplos usados para entrenar:  1920
Ejemplos usados para evaluar:  480
Ejemplos usados para test:  600


,description,label
2093,Después de un inicio prometedor de su carrera ...,1
1250,Continúa nuestra aventura en el submarino Tram...,2
1662,Partidazo en Anfield para cerrar el 2023 en to...,1
2140,"Nikola Jokic es el mejor jugador del mundo, o,...",1
371,El Solomillo de Cerdo es una carne que admite ...,0


A continuación preparamos los conjuntos de entrenamiento, validación y evaluación en **Pytorch** para poder hacer el entrenamiento del modelo.

In [9]:
import torch
import datasets

# Los datasets se preparan de manera distinta a Tensorflow, por lo
# que tendremos que crear la siguiente clase para su uso con PyTorch
class TGINEDataset(torch.utils.data.Dataset):
    # Constructor de la clase
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    # Funcion para devolver un item del conjunto de datos dado un índice
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    # Función para devolver la longitud del conjunto de datos
    def __len__(self):
        return len(self.labels)

import numpy as np
from datasets import load_metric

# Utilizaremos como métrica la precisión
metric = load_metric("accuracy")

# Función para
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

from transformers import TrainingArguments

# Definimos el batch size tanto para entrenamiento como para validación
batch_train_size = 16
batch_val_size = 64

# Definimos algunos training arguments como el tamaño del bach_size
training_args = transformers.TrainingArguments (
  output_dir = './results',
  logging_dir = './logs',
  per_device_train_batch_size = batch_train_size,
  per_device_eval_batch_size = batch_val_size
)

# Realizamos la tokenización de cada conjunto de datos, para "train", "validation" y "test"
tokenized_train_dataset = tokenizer (df_training.description.tolist (),  truncation=True, padding = True)
tokenized_eval_dataset = tokenizer (df_eval.description.tolist (), truncation=True, padding = True)
tokenized_test_dataset = tokenizer (df_test.description.tolist (), truncation=True, padding = True)


# Como hemos hecho antes, las etiquetas deben ser numéricas para poder entrenar.
# Preparamos los 3 datasets para hacer el finetuning
train_dataset = TGINEDataset (tokenized_train_dataset, df_training.label.tolist())
eval_dataset = TGINEDataset (tokenized_eval_dataset, df_eval.label.tolist())
test_dataset = TGINEDataset (tokenized_test_dataset, df_test.label.tolist())

from transformers import Trainer

# Definimos el objeto para realizar el reentrenamiento (Fine-Tuning) del modelo con los argumentos de entrenamiento definidos,
# con los conjuntos de "train" y "validation" definidos, y con la función definida para computar las
# métricas
trainer = Trainer (
    model = bert_class_model_pytorch,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    compute_metrics = compute_metrics,
)

# Realizamos Fine-Tuning del modelo
trainer.train()

# Realizamos las predicciones sobre el conjunto de validación
print ("PREDICCIONES SOBRE EVAL")
bert_class_model_pytorch.eval ()
print (json.dumps (trainer.evaluate (), indent = 2))


# Guardamos el nuevo modelo reentrenado
modelo ='modeloReentrenadoPytorch'
bert_class_model_pytorch.save_pretrained (modelo)
tokenizer.save_pretrained (modelo)

# Realizamos las predicciones sobre el conjunto de evaluación
print ("PREDICCIONES SOBRE TEST")
predictions = trainer.predict (test_dataset)
print(json.dumps(predictions.metrics, indent = 2))

<ipython-input-9-b0c0022e7b1e>:26: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Step,Training Loss


PREDICCIONES SOBRE EVAL


{
  "eval_loss": 0.06337900459766388,
  "eval_accuracy": 0.9708333333333333,
  "eval_runtime": 15.762,
  "eval_samples_per_second": 30.453,
  "eval_steps_per_second": 0.508,
  "epoch": 3.0
}
PREDICCIONES SOBRE TEST
{
  "test_loss": 0.2578183710575104,
  "test_accuracy": 0.9166666666666666,
  "test_runtime": 19.4509,
  "test_samples_per_second": 30.847,
  "test_steps_per_second": 0.514
}
